In [12]:
import sqlite3
import pandas as pd
import re
import json
import numpy as np
import os
from os.path import dirname, abspath, join
pd.options.display.max_colwidth = 500

In [11]:
final_crawl_dir = join(dirname(dirname(os.getcwd())), 'data', 'final-crawl')
db_odin = join(final_crawl_dir, "odin.sqlite")
db_webtap = join(final_crawl_dir, "webtap.sqlite")
# con = sqlite3.connect('/Users/aruneshmathur/Downloads/crawl/sqlite.database')
con = sqlite3.connect(db_webtap)
segments = pd.read_sql_query('''SELECT visit_id,
                                       node_id,
                                       top,
                                       left,
                                       width,
                                       height,
                                       inner_text,
                                       time_stamp
                                  FROM SEGMENTS
                                 WHERE inner_text GLOB '*[0-9]*';
                                ''', con)

Change time_stamp type

In [3]:
segments['time_stamp'] = pd.to_datetime(segments['time_stamp'])

Shape of segments?

In [4]:
segments.shape

(1201362, 4)

Filter out segments with no numbers in their inner_text

In [5]:
def regex_filter(val):
    if val:
        mo = re.search('\d+', val)
        if mo:
            return True
        else:
            return False
    else:
        return False

segments_non_numeric = segments[segments['inner_text'].apply(regex_filter)]

In [6]:
segments_non_numeric.shape

(279975, 4)

In [7]:
segments_non_numeric['inner_text'].head()

14    The QL Active Corrosion Protection System help...
16                                             41101900
17                                               313,58
19    Engine/drive: D4-210A-A, D4-210A-B, D4-225A-B,...
23                                            35,48 EUR
Name: inner_text, dtype: object

Group by visit_id and node_id, exclude all groups with just 1 member

In [8]:
segments_grouped = segments_non_numeric.groupby(['visit_id', 'node_id']).filter(lambda x: len(x) > 2)

In [9]:
segments_grouped.shape

(132518, 4)

In [10]:
segments_grouped[['visit_id', 'node_id', 'time_stamp', 'inner_text']].sort_values(['node_id','time_stamp']).head(n=100)

,visit_id,node_id,time_stamp,inner_text
752268,3026,247511,2019-02-02 14:20:09.556,00
752279,3026,247511,2019-02-02 14:20:10.960,59
752288,3026,247511,2019-02-02 14:20:11.862,58
752300,3026,247511,2019-02-02 14:20:12.558,57
752305,3026,247511,2019-02-02 14:20:13.560,56
752308,3026,247511,2019-02-02 14:20:14.560,55
752316,3026,247511,2019-02-02 14:20:15.558,54
752323,3026,247511,2019-02-02 14:20:16.558,53
752328,3026,247511,2019-02-02 14:20:17.558,52
752331,3026,247511,2019-02-02 14:20:18.558,51


Manipulate the groups:

In [21]:
def countdown(df):
    vals = df.sort_values(by='time_stamp').time_stamp.tolist()
    nums = []

    for i in range(len(vals) - 1):
        diff = vals[i+1] - vals[i]
        nums.append(diff.total_seconds())
        
    std = np.std(nums)
    count_vals = df.inner_text.nunique()
    
    if std < 1.0 and len(df.index) > 5 and count_vals != 1:
        print df.inner_text
        return True
    else:
        return False

timers = segments_grouped[['visit_id', 'node_id', 'time_stamp', 'inner_text']].groupby(['visit_id', 'node_id']).filter(countdown)

2831                  1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife
3021        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3022        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3023        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3024        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3025        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3026        1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader 

In [22]:
timers.shape

(27864, 4)

,visit_id,node_id,time_stamp,inner_text
2831,11,1531829286,2019-02-02 08:55:02.050,1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife
3021,11,1531829286,2019-02-02 08:55:02.480,1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3022,11,1531829286,2019-02-02 08:55:02.598,1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3023,11,1531829286,2019-02-02 08:55:02.642,1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson
3024,11,1531829286,2019-02-02 08:55:02.648,1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife\n1\nDyson


In [23]:
sorted(timers.visit_id.unique())

[11,
 24,
 31,
 40,
 42,
 72,
 75,
 89,
 121,
 190,
 196,
 200,
 263,
 266,
 285,
 314,
 373,
 382,
 400,
 407,
 431,
 495,
 617,
 620,
 666,
 678,
 686,
 687,
 694,
 696,
 700,
 730,
 738,
 770,
 776,
 833,
 860,
 909,
 924,
 950,
 972,
 974,
 981,
 993,
 996,
 1000,
 1016,
 1033,
 1057,
 1095,
 1103,
 1161,
 1162,
 1179,
 1185,
 1193,
 1206,
 1236,
 1302,
 1314,
 1368,
 1369,
 1374,
 1417,
 1419,
 1427,
 1441,
 1443,
 1459,
 1481,
 1565,
 1572,
 1579,
 1644,
 1669,
 1708,
 1724,
 1731,
 1757,
 1775,
 1782,
 1877,
 1941,
 2013,
 2017,
 2103,
 2104,
 2124,
 2169,
 2175,
 2186,
 2189,
 2196,
 2210,
 2260,
 2305,
 2308,
 2312,
 2317,
 2370,
 2397,
 2448,
 2459,
 2517,
 2539,
 2568,
 2575,
 2579,
 2587,
 2619,
 2633,
 2639,
 2684,
 2692,
 2702,
 2704,
 2724,
 2741,
 2798,
 2854,
 2855,
 2856,
 2859,
 2879,
 2891,
 2948,
 2957,
 2981,
 3026,
 3029,
 3118,
 3167,
 3168,
 3176,
 3194,
 3208,
 3209,
 3213,
 3268,
 3318,
 3343,
 3417,
 3449,
 3477,
 3488,
 3515,
 3527,
 3647,
 3683,
 3688,
 37

In [24]:
len(timers.visit_id.unique())

192

In [26]:
timers.groupby(['visit_id', 'node_id']).first()

time_stamp  \
visit_id node_id                              
11       1531829286 2019-02-02 08:55:02.050   
24       1173594903 2019-02-02 08:56:29.752   
31       1975739374 2019-02-02 08:57:21.766   
40       701989246  2019-02-02 08:58:28.194   
42       1088959191 2019-02-02 08:58:16.574   
         4198035516 2019-02-02 08:58:16.500   
72       1708603053 2019-02-02 09:01:49.748   
         2233665595 2019-02-02 09:01:49.748   
75       1564081513 2019-02-02 09:02:06.338   
         3472131378 2019-02-02 09:02:05.036   
89       597049561  2019-02-02 09:03:37.590   
121      617224042  2019-02-02 09:07:16.358   
190      1075331156 2019-02-02 09:14:40.478   
196      2170986833 2019-02-02 09:15:34.720   
200      2297366108 2019-02-02 09:15:33.212   
263      151426085  2019-02-02 09:22:30.974   
266      2363949188 2019-02-02 09:22:59.534   
285      3438156535 2019-02-02 09:24:52.390   
314      34669015   2019-02-02 09:28:12.284   
         159487238  2019-02-02 09:28:10.986   
         3078232844 2019-02-02 09:28:10.986   
373      290684133  2019-02-02 09:34:34.462   
         469606617  2019-02-02 09:34:34.526   
         1669694031 2019-02-02 09:34:34.526   
         2727845690 2019-02-02 09:34:34.462   
382      2647372576 2019-02-02 09:35:27.274   
400      3141737123 2019-02-02 09:37:27.570   
407      1322119913 2019-02-02 09:38:09.802   
         2311657612 2019-02-02 09:38:09.802   
431      2872567246 2019-02-02 09:40:39.568   
...                                     ...   
4294     1333842333 2019-02-02 16:37:18.898   
4358     1454094679 2019-02-02 16:44:09.536   
4383     4153408685 2019-02-02 16:46:54.426   
4384     1764658807 2019-02-02 16:46:56.880   
4452     1615372518 2019-02-02 16:54:16.254   
4475     123301380  2019-02-02 16:56:43.044   
4485     2665954687 2019-02-02 16:57:41.886   
4520     2724105648 2019-02-02 17:01:29.696   
4557     1159035900 2019-02-02 17:05:35.146   
4593     505882115  2019-02-02 17:09:29.742   
         1967602652 2019-02-02 17:09:29.742   
4594     3192708429 2019-02-02 17:09:45.126   
4666     2873350162 2019-02-02 17:17:41.118   
4704     2546412654 2019-02-02 17:21:43.490   
4709     476093158  2019-02-02 17:22:09.964   
         628410480  2019-02-02 17:22:09.964   
         1441776689 2019-02-02 17:22:09.964   
         3314498226 2019-02-02 17:22:09.964   
4724     1523671863 2019-02-02 17:23:41.998   
4728     3845205421 2019-02-02 17:24:22.448   
4778     4084343416 2019-02-02 17:29:44.090   
4821     3549932960 2019-02-02 17:34:20.204   
4824     3224738142 2019-02-02 17:34:44.496   
4834     3925730638 2019-02-02 17:35:42.776   
4925     1931598014 2019-02-02 17:45:15.220   
4948     1125351044 2019-02-02 17:48:01.108   
4961     291320640  2019-02-02 17:49:08.668   
4963     1108695046 2019-02-02 17:49:20.858   
4994     1305717054 2019-02-02 17:52:39.464   
         2779233182 2019-02-02 17:52:39.464   

                                                                                                                                                                                                                                                                              inner_text  
visit_id node_id                                                                                                                                                                                                                                                                          
11       1531829286                                                                                                                                     1\nDyson\n2\nNatraCare\n3\nLip bang\n4\nBetter Life\n5\nOXO\n6\nStep 2\n7\nTrader Joe's\n8\nCuraprox\n9\nSwimline\n10\nSunnyLife  
24       1173594903                                                                                                                                                                                                           